# Sentence Clustering  

문장으로 구성된 Raw Data에서 의미가 비슷한 것들을 클러스터로 묶어 분류하는 작업을 진행할 예정이다.
이 작업이 어떤 과정으로 이루어지는 필요한 개념과 도구를 알아본다.

러프하게 보면 각 문장들을 수치화하고 이 수치를 기반으로, 유사한 값들을 가진 것들끼리 묶는 과정으로 진행된다.

## DataSet 가져오기  

Dataset은 Google BigQuery Public Data Set을 이용한다.

> [Google Cloud Public Datasets](https://cloud.google.com/public-datasets/)


In [ ]:
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()

project_id = 'bigquery-public-data'

# # Create a "Client" object
client = bigquery.Client(project=project_id)

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "commits" table
table_ref = dataset_ref.table("commits")

# API request - fetch the table
table = client.get_table(table_ref)

# Extract commit message, convert to dataframe
dataframe = client.list_rows(table, 
                 selected_fields=[
                      bigquery.SchemaField("subject", "STRING")            
                 ],
                 max_results=1000).to_dataframe()

commitMessages = dataframe['subject']

## Embedding  

Embedding의 개념은 표현하고자 하는 대상을 dense vector 형태로 변환하는 것을 말한다. 
그 대상은 단어, 문장, 문서 전체, 이미지 등이 될 수 있다.  

여기서는 문장을 임베딩하는 Universal Sentence Encoder를 사용한다. 
입력으로 텍스트가 주어지고 출력으로는 512차원의 vector가 생성된다. 
google에서 만들었으며 tf_hub에 공개되어있다.

> [Universal Sentence Encoder](https://arxiv.org/abs/1803.11175)

In [ ]:
!pip install tensorflow_text

import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
embeddings = embed(commitMessages)

print(embeddings)

## K-Means Clustering  

수치화된 문장 즉, 임베딩 결과를 가지고 K-Means로 클러스터링한다.

In [ ]:
CLUSTER_CNT = 10

kmeans = KMeans(n_clusters = CLUSTER_CNT)
kmeans.fit(embeddings)

labels = kmeans.labels_
labelDict = dict()

# key : cluster number
# value : commit message array
for i, commitMessage in enumerate(commitMessages):
  if labels[i] in labelDict.keys():
    labelDict[labels[i]].append(commitMessage)
  else:
    labelDict[labels[i]] = [commitMessage]

for label in labelDict.keys():
  for commitMessage in labelDict[label]:
    print('{} : {}'.format(label, commitMessage))

## Elbow Method  

K-Means의 한계점으로는 클러스터 개수 선정에 있다.  
위의 예제에서는 10개의 클러스터로 분류를 하도록 진행하였는데, 
적절한 클러스터 개수를 어느정도 파악할 수 있는 elbow method을 적용해본다.  

그래프가 많이 꺾이면서 이 후 부터는 값의 변화가 거의 없는 지점을 elbow라고 한다. 
이 근처의 값을 선택하는 것이 어느정도 최적화 된 클러스터의 수라고 할 수 있다.

In [ ]:
from matplotlib import pyplot as plt

sse = []
for cluster_cnt in range(1, 20):
  kmeans = KMeans(n_clusters=cluster_cnt, init='k-means++', random_state=0)
  kmeans.fit(embeddings)
  sse.append(kmeans.inertia_)

plt.plot(range(1, 20), sse, marker='o')
plt.xlabel('cluster cnt')
plt.ylabel('SSE')
plt.show()